# Entorno

In [ ]:
import torch
import torchaudio
import numpy as np
import torch.nn as nn
from random import randint
import pytorch_lightning as pl
from pydub import AudioSegment
from scripts.extract import load_heart_noised_paths
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau
from scripts.plot import plot_audio_sample, plot_prediction_from_dataset
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    LearningRateMonitor,
)

# Relación de datos

In [ ]:
heart_noised = load_heart_noised_paths(
    # clean_dir="data/heart_sound_test_small",
    # noised_dir="data/heart_noised_test_small",
    clean_dir="data/heart_sound",
    noised_dir="data/heart_noised",
)

heart_noised[:3]

# Cómo suena?

In [ ]:
test = heart_noised[randint(0, len(heart_noised))]

audio_clean = AudioSegment.from_file(test[0])
audio_noisy = AudioSegment.from_file(test[1])

In [ ]:
audio_clean

In [ ]:
audio_noisy

# Cómo se ve?

In [ ]:
clean_array = np.array(audio_clean.get_array_of_samples())
noisy_array = np.array(audio_noisy.get_array_of_samples())
clean_array.shape, noisy_array.shape

In [ ]:
plot_audio_sample(clean_array, "Audio Limpio")
plot_audio_sample(noisy_array, "Audio con Ruido")

# CustomDataset class

In [ ]:
class CoraDenoiserDataset(Dataset):
    def __init__(self, data_tuples, transform=None, duration=None):
        self.data_tuples = data_tuples
        self.transform = transform
        if duration is None:
            self.min_duration = self.calculate_min_duration(level=10)
        else:
            self.min_duration = duration

    def __len__(self):
        return len(self.data_tuples)

    def calculate_min_duration(self, level: int = 1):
        self.durations = []
        for _, audio_path in self.data_tuples:
            audio, _ = torchaudio.load(audio_path)
            self.durations.append(audio.shape[-1])

        min_dur = min(self.durations) // level
        return min_dur * level

    def adjust_audio_duration(self, audio, duration):
        if audio.shape[-1] > duration:
            audio = audio[..., :duration]
        return audio

    def normalize_audio(self, audio):
        normalized = (audio - audio.mean()) / audio.std()
        return normalized

    def preprocess_audio(self, idx, is_clean):
        subidx = 0 if is_clean else 1
        audio_path = self.data_tuples[idx][subidx]
        audio, sample_rate = torchaudio.load(audio_path)
        audio = self.adjust_audio_duration(audio, self.min_duration)
        audio = self.normalize_audio(audio)
        return audio, sample_rate

    def __getitem__(self, idx):
        clean_audio, sample_rate = self.preprocess_audio(idx, is_clean=True)
        noisy_audio, _ = self.preprocess_audio(idx, is_clean=False)

        if self.transform:
            clean_audio = self.transform(clean_audio, sample_rate)
            noisy_audio = self.transform(noisy_audio, sample_rate)

        return clean_audio, noisy_audio

In [ ]:
from scripts.transform import spec_n_mfcc

denoiser_dataset = CoraDenoiserDataset(
    data_tuples=heart_noised,
    transform=spec_n_mfcc,
    duration=None
)

print("Pairs of sounds: ", len(denoiser_dataset))
print("Min audio duration: ", denoiser_dataset.min_duration)
denoiser_dataset[5][0].shape, denoiser_dataset[5][1].shape

In [ ]:
clean = denoiser_dataset[5][0][..., :denoiser_dataset.min_duration]
clean.shape, clean.mean(), clean.std()

In [ ]:
noisy = denoiser_dataset[5][1][..., :denoiser_dataset.min_duration]
noisy.shape, noisy.mean(), noisy.std()

# Arquitectura

In [ ]:
# class DownConvBlock(pl.LightningModule):
#     def __init__(
#         self,
#         input_size,
#         output_size,
#         kernel_size=3,
#         stride=2,
#         padding=1,
#         norm=True,
#         dropout=0.0,
#     ):
#         super(DownConvBlock, self).__init__()

#         self.layers = nn.Sequential(
#             nn.Conv1d(input_size, output_size, kernel_size, stride, padding),
#             nn.BatchNorm1d(output_size) if norm else nn.Identity(),
#             nn.LeakyReLU(0.2),
#             nn.Dropout1d(dropout) if dropout else nn.Identity(),
#         )

#     def forward(self, x):
#         output = self.layers(x)
#         return output


# class UpConvBlock(pl.LightningModule):
#     def __init__(
#         self, input_size, output_size, kernel_size=4, stride=2, padding=1, dropout=0.0
#     ):
#         super(UpConvBlock, self).__init__()

#         self.layers = nn.Sequential(
#             nn.ConvTranspose1d(input_size, output_size, kernel_size, stride, padding),
#             nn.BatchNorm1d(output_size),
#             nn.ReLU(),
#             nn.Dropout1d(dropout) if dropout else nn.Identity(),
#         )

#     def forward(self, x, encod_input):
#         x = self.layers(x)
#         output = torch.cat((x, encod_input), dim=1)
#         return output

In [ ]:
# class CoraDenoiser(pl.LightningModule):
#     def __init__(self, in_channels=1, out_channels=1):
#         super().__init__()
#         self.down_conv1 = DownConvBlock(in_channels, 16, norm=False)
#         self.down_conv2 = DownConvBlock(16, 32)
#         self.down_conv3 = DownConvBlock(32, 64)

#         self.up_conv1 = UpConvBlock(64, 32, dropout=0.2)
#         self.up_conv2 = UpConvBlock(64, 16, dropout=0.2)

#         self.upsample = nn.Upsample(scale_factor=2)
#         self.just_conv = nn.Conv1d(32, out_channels, kernel_size=1, padding=0)
    
#     def forward(self, x):
#         print("\n\nx", x.shape)
#         enc1 = self.down_conv1(x)
#         print("enc1", enc1.shape)
#         enc2 = self.down_conv2(enc1)
#         print("enc2", enc2.shape)
#         enc3 = self.down_conv3(enc2)
#         print("enc3", enc3.shape)

#         dec1 = self.up_conv1(enc3, enc2)
#         print("dec1", dec1.shape)
#         dec2 = self.up_conv2(dec1, enc1)
#         print("dec2", dec2.shape)

#         upsample = self.upsample(dec2)
#         print("upsample", upsample.shape)
#         final = self.just_conv(upsample)
#         print("final", final.shape, "\n")
        
#         return final

#     def training_step(self, batch, batch_idx):
#         clean, noisy = batch
#         # Forward pass
#         decoded = self(noisy)
#         # Calculamos la pérdida (error de reconstrucción)
#         train_loss = nn.MSELoss()(decoded, clean)
#         # Registramos la pérdida para su monitoreo
#         self.log("train_loss", train_loss, on_epoch=True)

#         return train_loss

#     def validation_step(self, batch, batch_idx):
#         clean, noisy = batch
#         decoded = self(noisy)
#         val_loss = nn.MSELoss()(decoded, clean)
#         self.log("val_loss", val_loss, on_epoch=True)

#         return val_loss

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
#         # optimizer = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)

#         # scheduler = {
#         #     "scheduler": CyclicLR(
#         #         optimizer, base_lr=0.001, max_lr=0.01, cycle_momentum=False
#         #     ),
#         #     "interval": "step",  # Frecuencia de ajuste del LR scheduler (en cada paso)
#         # }

#         # scheduler = {
#         #     "scheduler": ReduceLROnPlateau(optimizer, patience=3),
#         #     "monitor": "val_loss",  # Métrica para monitorear
#         #     "interval": "epoch",    # Frecuencia de ajuste del LR scheduler
#         #     "frequency": 1          # Igual a interval, ya que estamos usando "epoch"
#         # }

#         # return [optimizer], [scheduler]
#         return optimizer

In [ ]:
class CoraDenoiser(pl.LightningModule):
    def __init__(self, input_length, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_length, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_length),x
        )
    
    def forward(self, x):
        # print('x', x.shape)
        latent_code = self.encoder(x)
        # print('latent_code', latent_code.shape)
        reconstructed = self.decoder(latent_code)
        # print('reconstructed', reconstructed.shape)
        return reconstructed
    
    def training_step(self, batch, batch_idx):
        clean, noisy = batch
        # Forward pass
        decoded = self(noisy)
        # Calculamos la pérdida (error de reconstrucción)
        train_loss = nn.MSELoss()(decoded, clean)
        # Registramos la pérdida para su monitoreo
        self.log("train_loss", train_loss, on_epoch=True)

        return train_loss

    def validation_step(self, batch, batch_idx):
        clean, noisy = batch
        decoded = self(noisy)
        val_loss = nn.MSELoss()(decoded, clean)
        self.log("val_loss", val_loss, on_epoch=True)

        return val_loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Entrenamiento, validación y prueba

In [ ]:
batch_size = 30
train_ratio = 0.7
val_ratio = 0.15

train_size = int(train_ratio * len(denoiser_dataset))
val_size = int(val_ratio * len(denoiser_dataset))
test_size = len(denoiser_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    denoiser_dataset, [train_size, val_size, test_size]
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,
)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

# Callbacks

In [ ]:
checkpoint = ModelCheckpoint(
    monitor="val_loss",
    dirpath="checkpoints/",
    filename="autoencoder-{epoch:02d}-{val_loss:.4f}",
    save_top_k=1,
    mode="min",
)

early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")

lr_monitor = LearningRateMonitor(logging_interval="epoch")

# callbacks = [checkpoint, lr_monitor]
callbacks = [checkpoint, early_stopping, lr_monitor]

# Entrenamiento

In [ ]:
model = CoraDenoiser(
    input_length=denoiser_dataset[0][0].shape[-1],
    latent_dim=64
)

logger = TensorBoardLogger("logs/", name="cora_cleaner")

trainer = pl.Trainer(max_epochs=20, callbacks=callbacks, logger=logger)
trainer.fit(model, train_dataloader, val_dataloader)

# Predicción

In [ ]:
plot_prediction_from_dataset(
    model=model,
    dataset=denoiser_dataset,
    duration=denoiser_dataset.min_duration,
    idx=randint(0, len(denoiser_dataset))
)

In [ ]:
# Descomentar para revisar el Tensorboard en web
# !tensorboard --logdir=path_to_logs_directory

# Otra opción es VisualStudioCode: Ctrl+Shift+P -> Launch Tensorboard